# This notebook adds scores and labels to the discovery dataset
- score miRNA target genes as signatures
- annotate the dataset with KRAS mutation information

In [23]:
import scanpy as sc
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sn 
import sys
import os

In [24]:
import calculation_function as calc

In [25]:
sc.set_figure_params(figsize=[5,5])

## Load related data

In [7]:
#path to the scRNA h5ad files
data_path = "/home/lucy/disc_vali_full_dataset/"

In [3]:
#miRNA target gene file
miRNA_family = pd.read_csv("./converted_miRNA_family.csv", index_col=0)

In [4]:
miRNA_family.head()

,miR-10-5p,miR-135ab-5p,miR-218-5p,miR-190-5p,miR-193-3p,miR-33-5p,miR-182-5p,miR-362-5p,miR-23-3p,miR-18-5p,...,miR-361-5p,let-7d-3p,miR-375-3p,miR-148-3p/152-3p,miR-153-3p,miR-146ab-5p,miR-31-5p,miR-132-3p/212-3p,miR-99-5p/100-5p,miR-28-5p/708-5p
0,MCU,LTV1,PHF3,TP53INP1,TFCP2L1,ZBTB41,CTDSP1,DLL1,SGPP2,PRDX6,...,BLMH,PSMB4,RGS16,MGAT5,DOT1L,ATF6,MRPL44,VIL1,SCML4,KRT20
1,ZNF274,RNF167,ICOSLG,WDR11,SLC40A1,ATP10B,MYO1A,DNAJC5,NUAK2,MICU1,...,SLFN12L,UBXN1,EIF4EBP2,DEDD,ATP5O,ELMSAN1,SATB2,MAP7,TRIB2,DDX5
2,ZNF367,HNRNPAB,MED1,DCBLD1,LAMC2,JARID2,SKP1,TRPC4AP,PPP1R15B,SREBF1,...,HIF1A,CCNL1,HPCAL1,STX7,GAPDHS,ZNF367,MYO19,FOXO3,LPIN1,DCP1A
3,GNL3,SSR1,TOB2,CREBBP,BAZ2A,PIM3,MYO19,CA1,SCYL3,FASN,...,LARP4B,PLXNA1,NIPBL,AP1B1,NFAT5,ERBB2IP,LMO7,R3HDM4,APP,RAD21
4,FADS2,IFNGR2,SON,IMPAD1,MLLT6,PIM1,SPAG9,CCNE2,XKR4,TMEM170B,...,MAL2,CUL4B,TAT,SYNGR2,FBXO31,APPL1,PHF20L1,NEDD1,TGFBI,SLC3A2


In [5]:
miRNA_target_list = miRNA_family.values.flatten() #make a list of all miRNA target 

In [6]:
# a list of all target genes
miRNA_target_list = pd.Series(miRNA_target_list).dropna().unique()
miRNA_target_list.shape
type(miRNA_target_list)

numpy.ndarray

In [10]:
#discovery dataset
disc = sc.read(os.path.join(data_path, "VUMC_HTAN_DIS_EPI_V2.h5ad" ))

In [11]:
disc.raw = disc #save the raw count layer

In [12]:
calc.normalization(disc) #inplace normalization 

AnnData object with n_obs × n_vars = 65088 × 35272
    obs: 'HTAN Parent Data File ID', 'HTAN Specimen ID', 'Cell_Type', 'Polyp_Type', 'Sample_Classification'
    var: 'mean', 'std'
    obsm: 'X_pca', 'X_umap'

In [13]:
disc.X.sum(axis = 1)
#check normalization status 

array([-975.4978 , -844.81464, -276.75848, ..., -266.5004 , 1126.9031 ,
        411.42285], dtype=float32)

## Score miRNA target genes 

In [ ]:
#this loop scores each miRNA family's target gene expression

for i in range(len(miRNA_family.columns)):
    fam_name = miRNA_family.columns[i]
    sc.tl.score_genes(disc, gene_list=miRNA_family[fam_name].dropna(), 
                      ctrl_size=1000, use_raw=False, score_name=fam_name)




In [ ]:
#score all miRNA target genes as one score 
#store as 'all_targets'
sc.tl.score_genes(disc, gene_list= miRNA_target_list, ctrl_size=1000, use_raw=False, score_name="all_targets")

In [ ]:
#visualize the target gene score for one family
sc.pl.umap(disc, color = miRNA_family.columns[60], use_raw = False, vmax = 5, ncols = 4)

## KRAS mutation 

#### Getting KRAS mutation sample profile from the paper 

In [45]:
#adding KRAS mutation information to samples 

#see Chen et al., 2021, Cell 184, 6262–6280, https://doi.org/10.1016/j.cell.2021.11.031
#Figure 1C


KRAS_mut = []
num = 0
for i in range(disc.n_obs):
    if(disc.obs["Patient"][i] == "HTA11_866" or disc.obs["Patient"][i] == "HT11_5212"):
        num+=1
        KRAS_mut.append("missense")
    elif(disc.obs["Patient"][i] == "HTA11_1938"):
        KRAS_mut.append("multi-hit")
        num+=1
    else:
        KRAS_mut.append("no")

print(num)

5048


In [46]:
disc.obs["KRAS_mutation"] = KRAS_mut

In [22]:
disc.obs["KRAS_mutation"].unique()

['no', 'multi-hit', 'missense']
Categories (3, object): ['no', 'multi-hit', 'missense']

#### Specify KRAS multi-hit

In [30]:
kras_multi_col = []
for i in range(disc.n_obs):
    if disc.obs["KRAS_mutation"][i] == 'multi-hit':
        kras_multi_col.append("True")
    else:
        kras_multi_col.append("False" )
print(f"kras multi column correct length: {len(kras_multi_col) == disc.n_obs}")

kras multi column correct length: True


In [31]:
disc.obs["KRAS Multi-hit"] = kras_multi_col

## Save the header layer

In [17]:
#save the scores and other information columns to a new h5ad file 
header= disc
del header.X # do not save the count matrix

In [33]:
header.write("./disc_header.h5ad", compression='gzip')

... storing 'KRAS Multi-hit' as categorical


In [34]:
#